In [ ]:
from mymath import *
import torch
# import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
import plotly.graph_objects as go

In [ ]:
# Visualization of cubic spline

x = torch.linspace(-2.5, 2.5, 500)
y = spline3(x)

fig = go.Figure(data=[
    go.Scatter(x=x, y=y)
    ], layout=go.Layout(width=300, height=300))
fig.update_yaxes(exponentformat = 'E')
fig.show()

In [ ]:
smooth_factor = 2

def point_edge_potential_exact_integral(p, e0, e1, eps, r, alpha, n_samples):
    tangent = F.normalize(e1 - e0, p=2, dim=None)
    uv = torch.linspace(0, 1, n_samples)
    diff = (e0 - p).view(-1, 1) + torch.ger(e1 - e0, uv)
    dists = torch.norm(diff, dim=0)
    barrier_val = inv_barrier(dists, eps, r)
    Phi_val = spline3((tangent.view(1, 2) @ diff) / dists * (2 / alpha))
    return uv, barrier_val * Phi_val.view(-1)

def point_edge_potential_discrete(p, e0, e1, eps, r, alpha):
    tangent = e1 - e0
    length = torch.norm(tangent)
    normal = F.normalize(torch.tensor([-tangent[1], tangent[0]]), p=2, dim=None)

    # uv of closest point on extended edge
    s = torch.dot(tangent, p - e0).item() / length**2
    L = L_ns(s)**smooth_factor
    # smooth closest point
    y = e0 + tangent * (s - L)

    Phi = torch.dot(F.normalize(y - p, p=2, dim=None), tangent) ** 2 / length**2
    dist_sqr = torch.dot(p - e0, normal)**2 + length**2 * L**2
    
    return torch.norm(e1 - e0) * spline3(Phi * (2 / alpha)) * inv_barrier(dist_sqr, eps, r)


In [ ]:
# Visualization of integrand in Psi(x) over an edge

e0 = torch.tensor([0., 0.])
e1 = torch.tensor([0., 1.])
p = torch.tensor([0.1, 0.6])

eps = 5e-1
r = 1
alpha = 2
n_samples = 10000

uv, integrand = point_edge_potential_exact_integral(p, e0, e1, eps, r, alpha, n_samples)

fig = go.Figure(data=[
    go.Scatter(x=uv, y=integrand)
    ], layout=go.Layout(width=400, height=400))
fig.update_yaxes(exponentformat = 'E')
fig.show()

In [ ]:
# potential field in the space

def percentile(L, ratio=0.999):
    X = np.sort(L.flatten())
    id = int(len(X)*ratio)
    return X[id]

dhat = 0.5
r = 1
alpha = 1
e0 = torch.tensor([-1, 0.])
e1 = torch.tensor([0, 0.])
xs = torch.linspace(-1.1, 0.1, 400)
ys = torch.linspace(-0.02, 0.02, 80)

distances = torch.zeros((ys.shape[0], xs.shape[0]))
for i, x in enumerate(xs):
    for j, y in enumerate(ys):
        distances[j, i] = point_edge_potential_discrete(torch.tensor([x, y]), e0, e1, dhat, r, alpha)

major_max = percentile(distances)
print("major range", [0, major_max])
fig = go.Figure(data=[
    go.Scatter(x=[e0[0], e1[0]], y=[e0[1], e1[1]]),
    go.Contour(z=distances, x=xs, y=ys, colorscale='Hot',
    contours=dict(
        start=0,
        end=major_max,
        size=major_max/10,
    ))
], layout=go.Layout(width=800, height=400))
fig.show()

In [ ]:
import meshplot as mp
from scipy.spatial.transform import Rotation
import ipywidgets as iw

e0 = torch.tensor([0., 0.])
e1 = torch.tensor([0., 1.])
p = torch.tensor([0.1, 0.6])

eps = 5e-1
r = 1
alpha = 2
n_samples = 10000

tb = iw.ToggleButtons(description='Segment', options=[1,'+'])
@mp.interact(x=tb)
def s_but(x):
    if x == '+':
        tb.options = list(range(1,len(tb.options)+1)) + ['+']
        tb.value = len(tb.options) - 1
    else:
        tb.current = x

button = iw.Button(description="Paint!")
clear_button = iw.Button(description="Clear!")

segments_recording = np.zeros(n_samples, dtype=int)
# Set Callback
def paint_clicked(b):
    slicer = np.where(np.linalg.norm(v - sf.coord[1:],axis=1) < sf.coord[0])[0]
    segments_recording[slicer] = tb.current
    paint_ui.update_object(oid=0, colors=segments_recording)
def clear_clicked(b):
    segments_recording[segments_recording == tb.current] = 0
    paint_ui.update_object(oid=0, colors=segments_recording)
button.on_click(paint_clicked)
clear_button.on_click(clear_clicked)

# Display Buttons
display(iw.HBox([button, clear_button]))

# Meshplot
paint_ui = mp.plot(v,f,c=segments_recording)
paint_ui.add_mesh(v_s*0.1, f_s,shading={"flat" : False},c=np.array([1,0,0]))
def sf(x,y):
    paint_ui.update_object(oid = 1, vertices = v_s*radius + np.array([x,y,z]))
    sf.coord = [x,y]
mp.interact(sf,
            x = iw.FloatSlider(min=-0.5, max=1.5, value=0.5),
            y = iw.FloatSlider(min=-0.5, max=0.5, value=0.01))